In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)

import os, torch
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

2024-08-01 07:25:50.982124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 07:25:50.982249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 07:25:51.112165: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "..." # Redacted as its trained on my personal chats

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format

tokenizer = AutoTokenizer.from_pretrained(base_model)

torch_dtype = torch.float16
attn_implementation = "eager"

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation=attn_implementation
)

base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [8]:
messages = [
    {
        "role": "user",
        "content": "Hey doctor, I have severe acne on my face, what to do?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

🥶 I m not a doctor🥹😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂


In [7]:
messages = [
    {
        "role": "user",
        "content": "Hey u doing good?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")
for i in range(3):
    
    outputs = model.generate(**inputs, max_length=150, 
                             num_return_sequences=1)

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f'Response {i+1}')
    print(text.split("assistant")[1])
    print('')

Response 1

I m Fine I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in pain I m in

Response 2

Fine 😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂�

Response 3

Yes of course how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r u how r



### Model does tend to give sensible responses but often generates gibberish/repeated sentences. This project is a proof of concept that such a model can be made, this models main limitations are:

- It's 4-bit quantized nature.

- It being finetuned on a language it is trained a very little on (Hindi) and a way of talking it was never trained on.

- Limited training data/computational resources.